<a href="https://colab.research.google.com/github/Luicazen/NLPTweets/blob/SVM/initSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as textb

In [31]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np

Saving train.csv to train (1).csv
Saving test.csv to test (1).csv


**Setting up the training data**




In [32]:
#Read in Training data
trainingDF = pd.read_csv('train.csv')
#Setting index to start at 1
trainingDF.index +=1
print("Training Data Summary")
#printing summary features: Shape
display(trainingDF.shape)
#Stats
display(trainingDF.describe())
#First 5 rows
display(trainingDF.head())
#type summary
trainingDF.info()
#column names
print(trainingDF.columns)

Training Data Summary


(7613, 5)

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


,id,keyword,location,text,target
1,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
2,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
3,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
4,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
5,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 1 to 7613
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [33]:
#Check for null data
print("Training Data")
trainingDF.isnull().sum()

Training Data


,0
id,0
keyword,61
location,2533
text,0
target,0


*As shown above, most NaN data is found in either the keyword or location feild.*

In [45]:
#drop the target column from the training data X
X = trainingDF.drop(['target'], axis=1)
#Y is only the target column
y = trainingDF['target']


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape, X_test.shape

((6090, 4), (1523, 4))

In [41]:
cols_to_scale = ['id', 'keyword', 'location']

scaler = StandardScaler()

X_train_scaled_part = scaler.fit_transform(X_train_encoded_non_text[cols_to_scale])
X_test_scaled_part = scaler.transform(X_test_encoded_non_text[cols_to_scale])

X_train_scaled_df = pd.DataFrame(X_train_scaled_part, columns=cols_to_scale, index=X_train_encoded_non_text.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled_part, columns=cols_to_scale, index=X_test_encoded_non_text.index)

X_train[cols_to_scale] = X_train_scaled_df
X_test[cols_to_scale] = X_test_scaled_df

X_train.shape, X_test.shape
X_train.describe()

,id,keyword,location
count,4096.000000,4096.000000,4096.000000
mean,-0.012536,-0.015542,-0.013025
std,1.001802,0.999323,1.004641
min,-1.731711,-1.712472,-1.601777
25%,-0.885553,-0.882399,-1.033364
50%,-0.023094,-0.036664,0.104872
75%,0.856516,0.856057,1.070527
max,1.731711,1.748777,1.070527


Benchmark F1 score for un-tuned SVM is: 0.3706